In [1]:
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch
import numpy as np
import textstat

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import numpy as np
from datetime import datetime
import yfinance as yf
import os
import glob
import regex as re
import csv
import statistics
import nltk
nltk.download('punkt')

sentiment_finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
sentiment_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

sentiment_nlp = pipeline("text-classification", model=sentiment_finbert, tokenizer=sentiment_tokenizer)
fls_finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3)
fls_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')
fls_nlp = pipeline("text-classification", model=fls_finbert, tokenizer=fls_tokenizer)

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import auth
import pickle

[nltk_data] Downloading package punkt to /Users/victor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open("info/rfc_banks_model.pkl", "rb") as f:
    banks_rfc = pickle.load(f)

with open("info/rfc_automobiles_model.pkl", "rb") as f:
    automobiles_rfc = pickle.load(f)

with open("info/rfc_consumer_durables_model.pkl", "rb") as f:
    consumer_durables_rfc = pickle.load(f)

In [3]:
# connect to firebase
cred = credentials.Certificate('info/computing-nea-a3b68-firebase-adminsdk-m2onq-44557de7c9.json')
app = firebase_admin.initialize_app(cred)

db = firestore.client()

In [ ]:
# current user emails and password:

# user 1:
# email = testing1@gmail.com
# password = test123


# user 2:
# email = testingperson2@gmail.com
# password = test456

In [4]:
def submit_user_details(user_id):
    year = int(input("What year were you born?"))
    month = int(input("What month were you born?"))
    day = int(input("What day were you born?"))

    dob = datetime(year=year, month=month, day=day)

    first_name = input("What's your first name?")
    last_name = input("What's your last name?")
    sex = input("What's your sex?")

    user_details = {
        'dob': dob,
        'first_name': first_name,
        'last_name': last_name,
        'sex': sex
    }

    doc_ref = db.collection(u'userdetails').document(user_id)
    doc_ref.set(user_details)

In [5]:
email = "testing1@gmail.com"
password = "test123"

In [24]:
# create a user account

# email = input("Enter your email: ")
# password = input("Enter your password: ")

try:
    user = auth.create_user(email=email, password=password)

except Exception as exception_message:
    print("There has been an error creating an account:", exception_message)

user_id = user.uid

print("User ID:", user_id)

submit_user_details(user.uid)

User ID: Yxn9r9bkK8fpxHEXgTMSo69oQGD2


TypeError: submit_user_details() missing 1 required positional argument: 'user_id'

In [6]:
# login user account


# email = input("Enter your email: ")
# password = input("Enter your password: ")

try:
    user = auth.get_user_by_email(email)
    print("You have successfully logged in!")
except Exception as exception_message:
    print("There has been an error signing in:", exception_message)

user_id = user.uid

print("User ID:", user_id)



You have successfully logged in!
User ID: Yxn9r9bkK8fpxHEXgTMSo69oQGD2


In [10]:
# Update user email
new_email = 'replaceEmail@gmail.com'
try:
    user = auth.update_user(
        user.uid,
        email=new_email,
    )
    print("You have successfully updated your email address")
except Exception as exception_message:
    print("There has been an error updating your account:", exception_message)



You have successfully updated your email address


In [28]:
# Update user details:
submit_user_details(user.uid)

In [10]:
# delete user account
AreYouSure = input("Are you sure you want to delete this account? -- Y or N")
if AreYouSure == "Y":
    try:
        auth.delete_user(user.uid)

        # also deletes it from the userdetails collection
        doc_ref = db.collection('userdetails').document(user.uid)
        doc_ref.delete()
        print("You have successfully deleted this account")
    except Exception as exception_message:
        print("There has been an error deleting your account:", exception_message)

else:
    print("You have not deleted this account")

You have not deleted this account


In [ ]:
stock = input("enter stock")
year = input("enter year")
quarter = input("enter quarter")

In [57]:
stock = "GM"
year = 2022
quarter = 4
document_name, stock_data, stock_data_with_date = find_stock_data(stock, year, quarter, user_id)

The stock you are searching is already in the global stockpresets collection, the program does not need to compute it again
Successfully updated the stock to your view history


In [26]:
print("Your current watchlists are:")
# from userdetails collection
user_ref = db.collection('userdetails').document(user_id)
collections = user_ref.collections()
for collection in collections:
    print(collection.id)

watchlist_name = input("Please enter a watchlist to add this stock to, or enter 'N' if you don't want to save it to your watchlists (the programme will automatically create a watchlist for you if it doesn't exist)")
if watchlist_name == "N":
    exit()
else:
    try:
        watchlist_ref = user_ref.collection(watchlist_name)
        stock_ref = watchlist_ref.document(document_name)
        stock_ref.set(stock_data_with_date)
        print(f'Successfully added {document_name} to the watchlist {watchlist_name}!')
    except:
        print("There has been an error with adding to your watchlist:", exception_message)
    


Your current watchlists are:
watchlist1
watchlist3
Successfully added GM20224 to the watchlist watchlist1!


In [56]:
def find_stock_data(stock, year, quarter, user_id):
    sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
    "consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
    "energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
    "pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
    "tech", "telecom", "transportation", "utilities"]

    for sectorin in sectorlist:
        filelist = os.listdir("sectors/"+sectorin)
        try:
            filelist.remove('.DS_Store')
        except:
            pass
        for stockin in filelist:
            if stockin == stock:
                sector = sectorin

    if sector == "banks":
        loaded_rfc = banks_rfc

    elif sector == "automobiles":
        loaded_rfc = automobiles_rfc

    elif sector == "consumer-durables":
        loaded_rfc  = consumer_durables_rfc

    else:
        print("this sector doesn't have a preset model")
        exit()
    
    document_name = stock + str(year) + str(quarter)
    doc_ref = db.collection(u'stockpresets').document(document_name)

    # if the stock data has already been logged inside the "stockpresets" collection, then it will fetch this data from firebase instead of computing everything all over again
    if doc_ref.get().exists:
        print("The stock you are searching is already in the global stockpresets collection, the program does not need to compute it again")

        stock_data = doc_ref.get().to_dict()

    else:
        print("The stock you are searching is not already in the global stockpresets collection, please wait until the program computes the results.")

        document_name = stock + str(year) + str(quarter)

        doc_ref = db.collection(u'stockpresets').document(document_name)

        ticker = yf.Ticker(stock)
        market_cap = ticker.fast_info['market_cap']
        mytranscript, exact_date = get_transcript(stock, year, quarter)
        earnings_list = ticker.get_earnings_dates(limit=30) #uses yfinance and gets historical eps consensus/reported data
        earnings_list.reset_index(inplace=True)
        earnings_list = earnings_list[earnings_list['Surprise(%)'].notna()]
        eps_list = earnings_list.reset_index(drop=True)
        for index in range(0, len(eps_list)):
            percentage_surprise = float(eps_list['Surprise(%)'][index])
            date = eps_list['Earnings Date'][index]
            date = date.replace(tzinfo=None)
            date = date.to_pydatetime()
            mycase = check_day_diff(date, exact_date)
            if mycase == True:
                myEPS = percentage_surprise
                break
        transcript_safe_harbour, transcript_questions = split_transcript(mytranscript)
        speaker_names = get_file_speaker_names(sector, stock)
        analyst_names = find_analyst_names(speaker_names, transcript_questions)
        analyst_sentences, management_sentences = get_analyst_management_sentences(analyst_names, transcript_questions)
        fea_ext_list5to22 = getFeature5to22(transcript_safe_harbour, speaker_names)
        fea_ext_list23to43 = getFeature23to43(analyst_sentences, management_sentences, speaker_names)
        fea_ext_list44to73 = getFeature44to73(mytranscript, speaker_names)
        X_dataset = [myEPS] + fea_ext_list5to22 + fea_ext_list23to43 + fea_ext_list44to73 + [market_cap] + [stock] + [year] + [quarter] + [sector]

        X_data = np.array(X_dataset)
        Single_X_data = X_data[:70]
        Single_X_data = Single_X_data.reshape(1,-1)


        Y_pred = loaded_rfc.predict(Single_X_data)

        Y_pred_proba = loaded_rfc.predict_proba(Single_X_data)

        if Y_pred == "bin_2":
            direction = "Up"

        elif Y_pred == "bin_1":
            direction = "Down"
            
        stock_data = {
            "stock": stock,
            "year": year,
            "quarter": quarter,
            "direction": direction,
            "probabilityUp": Y_pred_proba[0][1],
            "probabilityDown": Y_pred_proba[0][0]
        }

        doc_ref = db.collection(u'stockpresets').document(document_name)
        doc_ref.set(stock_data)
        print("The stock information that will be uploaded to the database is:", stock_data)
        print("Successfully added the contents to the global database")

    try:
        user_ref = db.collection('viewhistory').document(user_id)

        # checks if the userID document exists or not, if not, it will be created
        if not user_ref.get().exists:
            user_ref.set({})

        current_time = datetime.now()
        stock_view_filename = stock + current_time.strftime("%Y-%m-%d %H:%M:%S")
        
        # checks if the history collection exists
        history_ref = user_ref.collection('history')
        stock_ref = history_ref.document(stock_view_filename)
        
        stock_data_with_date = stock_data.copy()
        stock_data_with_date["DateViewed"] = current_time
        stock_ref.set(stock_data_with_date)

        print('Successfully updated the stock to your view history')
    except Exception as exception_message:
            print("There has been an error with updating your view history:", exception_message)
                
    return document_name, stock_data, stock_data_with_date

In [20]:
def check_day_diff(date1, exact_date):
    day_difference = abs((exact_date - date1).days)
    if day_difference < 5:
        return True
    else:
        return False

def find_transcript(stock, date_yr, date_qtr):
    try:
        url = 'https://roic.ai/transcripts/' + stock + '?y=' + str(date_yr) + '&q=' + str(date_qtr)
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, "html.parser")
        script = soup.find_all('script')[15].text.strip()  
        data = json.loads(script)
        transcript_data = data['props']['pageProps']['transcriptdata']['content']  # loads the transcript content
        json_dates = data['props']['pageProps']['data']['data']['earningscalls']
        for info in json_dates:
            if info["year"] == year and info["quarter"] == quarter:
                exact_date = datetime.strptime(info["date"], '%Y-%m-%d %H:%M:%S')
    except:
        print("sorry, the transcript cannot be found, please enter a different date")
        exit()
        
    return transcript_data, exact_date

def get_transcript(stock, year, quarter):
    mytranscript = ""
    mytranscript, exact_date = find_transcript(stock, year, quarter)
    mytranscript = re.sub(r'[^A-Za-z0-9.,:!\'\n ]', '', mytranscript)
    mytranscript = mytranscript.replace(".", ". ")
    mytranscript = re.sub('[^\S\n]+', ' ', mytranscript) #replaces multiple spaces to single space, without deleting newlines \n in the process
    mytranscript = mytranscript.splitlines() # finds transcript
    return mytranscript, exact_date

def split_transcript(mytranscript):    
    transcript_safe_harbour, transcript_questions = "", ""
    for i in range(0, len(mytranscript)):
        speech_bubble = mytranscript[i].lower()
        speech_bubble = re.sub(r'[^\w\s:]', ' ', speech_bubble) # regex: replaces all punctuations (except for ":") with 1 open space so the IF condition below can run smoothly
        if (i > 1) and (("operator:" in speech_bubble) and (("question" in speech_bubble) or ("go ahead" in speech_bubble) or ("operator instructions" in speech_bubble))):
            transcript_safe_harbour = mytranscript[0:i]
            transcript_questions = mytranscript[i:]
            break
        elif (i > 1 ) and ("operator" in speech_bubble) and ("question" in speech_bubble):
            transcript_safe_harbour = mytranscript[0:i+1]
            transcript_questions = mytranscript[i+1:]
            break
        elif (i > 1 ) and ("operator:" in speech_bubble) and ("first" in speech_bubble):
            transcript_safe_harbour = mytranscript[0:i]
            transcript_questions = mytranscript[i:]
            break

    return transcript_safe_harbour, transcript_questions

def get_file_speaker_names(sector, stock):
    write_path = "sectors/"+sector+"/"+stock+"/"+"speaker names.csv"
    speaker_names = np.loadtxt(write_path, delimiter='\t', dtype=str)
    return speaker_names
    
def find_analyst_names(speaker_names, transcript_questions):
    analyst_names = []
    for index in range(0, len(transcript_questions)-1):
        speech_bubble = transcript_questions[index].lower()
        speech_bubble = re.sub(r'[^\w\s:]', ' ', speech_bubble) # regex: replaces all punctuations (except for ":") with 1 open space 
        if ("operator:" in speech_bubble) or ("operator :" in speech_bubble):
            for name in speaker_names:
                namelist = name.split()
                if (name.lower() != "operator") and ("representative" not in name.lower()) and ("corporate" not in name.lower()) and ("company" not in name.lower()):
                    for name_2 in namelist: # cycle through each name in the name_list
                        name_2 = name_2.lower()
                        if (((" "+name_2+" " in speech_bubble) and len(name_2) > 2) and (("end" not in speech_bubble) and ("closing" not in speech_bubble) and (("turn" not in speech_bubble) or (("over" not in speech_bubble))))) and (name_2 in transcript_questions[index+1].lower()):
                            analyst_names.append(name)
                    if "unidentified" in name.lower().split(): # finds name such as "Unidentified Analyst"
                        analyst_names.append(name) 
                        
    analyst_names = list(set(analyst_names)) # replaces duplicates        
    return analyst_names

def get_analyst_management_sentences(analyst_names, transcript_questions):
    analyst_sentences = []
    management_sentences = []
    for index in range(0, len(transcript_questions)-1):
        speech_bubble = transcript_questions[index]
        colon_pos = speech_bubble.find(":")
        speaker_name = speech_bubble[:colon_pos]
        if index > 3:
            for name in analyst_names:
                namelist = name.split()
                if (speech_bubble not in analyst_sentences):
                    if speaker_name in analyst_names:
                        analyst_sentences.append(speech_bubble)

                    elif (speaker_name.lower() == "operator") or (speaker_name.lower() == "operator "):
                        pass

                    elif ((namelist[0] in speaker_name) or (namelist[-1] in speaker_name)) and (("operator:" in transcript_questions[index-1].lower()) or ("operator :" in transcript_questions[index-1].lower())):

                        analyst_names.append(speaker_name)
                        analyst_sentences.append(speech_bubble)

    # get management sentence
    for index in range(0, len(transcript_questions)-2):
        speech_bubble = transcript_questions[index]
        colon_pos = speech_bubble.find(":")
        speaker_name = speech_bubble[:colon_pos]
        
        # dont want operator's sentence 
        if (speaker_name.lower() == "operator") or (speaker_name.lower() == "operator "):
            pass
        
        elif (speech_bubble not in analyst_sentences):
            management_sentences.append(speech_bubble)
    return analyst_sentences, management_sentences

class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_word = False
        self.fail = None
        self.word = None

class AhoCorasick:
    def __init__(self, words):
        self.root = TrieNode()
        self.build_trie(words)
        self.build_ac_automata()

    def build_trie(self, words):
        for word in words:
            node = self.root
            for char in word:
                if char not in node.children:
                    node.children[char] = TrieNode()
                node = node.children[char]
            node.is_word = True
            node.word = word

    def build_ac_automata(self):
        queue = []

        for node in self.root.children.values():
            queue.append(node)
            node.fail = self.root

        while len(queue) > 0:
            node = queue.pop(0)
            for char, child in node.children.items():
                queue.append(child)
                fail_node = node.fail
                while fail_node is not None and char not in fail_node.children:
                    fail_node = fail_node.fail
                if fail_node is None:
                    child.fail = self.root
                else:
                    child.fail = fail_node.children[char]
                child.is_word |= child.fail.is_word

    def remove_words(self, text):
        node = self.root
        new_text = text
        for i, char in enumerate(text):
            while node is not None and char not in node.children:
                node = node.fail
            if node is None:
                node = self.root
                continue
            node = node.children[char]
            if node.is_word:
                new_text = new_text.replace(node.word, '')
        return new_text


def getFeature5to22(pre_release, speaker_names):
    new_speaker_names = [word + ':' for word in speaker_names]

    ac = AhoCorasick(new_speaker_names)

    net_sentiment_list = []
    flesch_list = []

    n_flslist = []
    s_flslist = []
    ns_flslist = []

    net_positive = 0
    net_negative = 0
    net_neutral = 0

    feature_extract_5 = 0
    feature_extract_6 = 0
    feature_extract_7 = 0
    feature_extract_8 = 0
    feature_extract_9 = 0
    feature_extract_10 = 0
    feature_extract_11 = 0
    feature_extract_12 = 0
    feature_extract_13 = 0
    feature_extract_14 = 0
    feature_extract_15 = 0
    feature_extract_16 = 0
    feature_extract_17 = 0
    feature_extract_18 = 0
    feature_extract_19 = 0
    feature_extract_20 = 0
    feature_extract_21 = 0
    feature_extract_22 = 0

    try:
        for speech_bubble in pre_release:
            try:
                new_speech_bubble = ac.remove_words(speech_bubble)
                new_speech_bubble = re.sub('[^\S\n]+', ' ', new_speech_bubble)

                if new_speech_bubble[0] == " ": 
                    new_speech_bubble = new_speech_bubble.replace(" ", "", 1) # replace the first space bar with an empty string, for example ' is is string is is string' to 'is is string is is string'
                    
                # gets text complexity
                flesch_score = textstat.flesch_reading_ease(new_speech_bubble)
                flesch_list.append(flesch_score)

                new_speech_bubble_list = split_paragraph_into_sentences(new_speech_bubble)

                fls_results = fls_nlp(new_speech_bubble_list)
                
                for i in range(0, len(new_speech_bubble_list)):
                    sentence = new_speech_bubble_list[i]
                    if fls_results[i]['label'] == 'Not FLS':
                        n_flslist.append(sentence)
                    elif fls_results[i]['label'] == 'Specific FLS':
                        s_flslist.append(sentence)
                    elif fls_results[i]['label'] == 'Non-specific FLS':                    
                        ns_flslist.append(sentence) 
                        
            except:
                pass

        try:
            feature_extract_9, feature_extract_10, feature_extract_11, feature_extract_12, fls1_sentiment_list, net1_positive, net1_negative, net1_neutral = get_fls_features(s_flslist)
            
        except:
            pass

        try:
            feature_extract_13, feature_extract_14, feature_extract_15, feature_extract_16, fls2_sentiment_list, net2_positive, net2_negative, net2_neutral = get_fls_features(ns_flslist)

        except:
            pass

        try:
            feature_extract_17, feature_extract_18, feature_extract_19, feature_extract_20, fls3_sentiment_list, net3_positive, net3_negative, net3_neutral = get_fls_features(n_flslist)

        except:
            pass

        try:
            numb_s_flslist = len(s_flslist)
            numb_ns_flslist = len(ns_flslist)
            numb_n_flslist = len(n_flslist)
            total = numb_s_flslist + numb_ns_flslist + numb_n_flslist

            feature_extract_21 = numb_s_flslist/total
            feature_extract_22 = numb_ns_flslist/total

        except:
            pass
        
        try:
            net_positive = net1_positive + net2_positive + net3_positive
            net_negative = net1_negative + net2_negative + net3_negative
            net_neutral = net1_neutral + net2_neutral + net3_neutral
            net_sentiment_list = fls1_sentiment_list + fls2_sentiment_list + fls3_sentiment_list
        except:
            pass

        try:
            feature_extract_5 = statistics.mean(net_sentiment_list)
        except:
            pass

        try:
            feature_extract_6 = net_positive/(net_negative+net_positive+net_neutral)
            feature_extract_7 = net_negative/(net_negative+net_positive+net_neutral)
        except:
            pass

        try:
            feature_extract_8 = statistics.mean(flesch_list)
        except:
            pass

        fea_ext_list5to22 = [feature_extract_5, feature_extract_6, feature_extract_7, feature_extract_8, feature_extract_9, feature_extract_10, feature_extract_11, feature_extract_12, feature_extract_13, feature_extract_14, feature_extract_15, feature_extract_16, feature_extract_17, feature_extract_18, feature_extract_19, feature_extract_20, feature_extract_21, feature_extract_22]

    except:
        fea_ext_list5to22 = [0]*18
    
    return fea_ext_list5to22


def get_fls_features(flslist):
    fls_sentiment_list = []
    net_positive = 0
    net_negative = 0
    net_neutral = 0
    
    feature_extract_1 = 0
    feature_extract_2 = 0
    feature_extract_3 = 0
    feature_extract_4 = 0

    for each_fls in flslist:
        sentiment_result = sentiment_nlp(each_fls)
        sentiment_score, positivity_value = map_sentiments(sentiment_result)
        fls_sentiment_list.append(sentiment_score)
        
        if positivity_value == "positive":
            net_positive += 1

        elif positivity_value == "negative":
            net_negative += 1

        else:
            net_neutral += 1
    try:
        feature_extract_1 = statistics.mean(fls_sentiment_list)
    except:
        pass
    try:
        feature_extract_2 = net_positive/(net_positive+net_negative+net_neutral)
    except:
        pass
    try:
        feature_extract_3 = net_negative/(net_positive+net_negative+net_neutral)
    except:
        pass
    try:
        feature_extract_4 = textstat.flesch_reading_ease(' '.join(flslist))
    except:
        pass

    return feature_extract_1, feature_extract_2, feature_extract_3, feature_extract_4, fls_sentiment_list, net_positive, net_negative, net_neutral


def getFeature23to43(analyst_speech, management_speech, speaker_names):
    new_speaker_names = [word + ':' for word in speaker_names]
    ac = AhoCorasick(new_speaker_names)

    n_flslist = []
    s_flslist = []
    ns_flslist = []

    questions_complex_list = []
    reply_complex_list = []
    net_text_complex_list  = []

    # list of sentiments for all S_FLS, N_FLS, NS_FLS classes
    s_fls_sentiment_list = []
    n_fls_sentiment_list = []
    ns_fls_sentiment_list = []

    # list of sentiments for all sentences that are identified as a "question"
    question_sentiment_list = []

    # list of sentiments for all sentences that are identified as a "reply"
    reply_sentiment_list = []

    # list of sentiments for all sentences in the Q&A section
    net_sentiment_list = []

    net_positive = 0
    net_negative = 0
    net_neutral = 0

    Qpositive = 0
    Qnegative = 0
    Qneutral = 0

    Rpositive = 0
    Rnegative = 0
    Rneutral = 0

    SFLSpositive = 0
    SFLSnegative = 0
    SFLSneutral = 0

    NSFLSpositive = 0
    NSFLSnegative = 0
    NSFLSneutral = 0

    NFLSpositive = 0
    NFLSnegative = 0
    NFLSneutral = 0 

    feature_extract_23 = 0
    feature_extract_24 = 0
    feature_extract_25 = 0
    feature_extract_26 = 0

    feature_extract_27 = 0
    feature_extract_28 = 0
    feature_extract_29 = 0
    feature_extract_30 = 0

    feature_extract_31 = 0
    feature_extract_32 = 0
    feature_extract_33 = 0
    feature_extract_34 = 0

    feature_extract_35 = 0
    feature_extract_36 = 0
    feature_extract_37 = 0
    
    feature_extract_38 = 0
    feature_extract_39 = 0
    feature_extract_40 = 0
    
    feature_extract_41 = 0
    feature_extract_42 = 0
    feature_extract_43 = 0

    try:
        for speech_bubble in analyst_speech:
            try:
                new_speech_bubble = ac.remove_words(speech_bubble)
                new_speech_bubble = re.sub('[^\S\n]+', ' ', new_speech_bubble)

                if new_speech_bubble[0] == " ": 
                    new_speech_bubble = new_speech_bubble.replace(" ", "", 1) # replace the first space bar with an empty string, for example ' is is string is is string' to 'is is string is is string'
                    
                # gets text complexity
                flesch_score = textstat.flesch_reading_ease(new_speech_bubble)
                questions_complex_list.append(flesch_score)
                net_text_complex_list.append(flesch_score)

                new_speech_bubble_list = split_paragraph_into_sentences(new_speech_bubble)

                for i in range(0, len(new_speech_bubble_list)):
                    sentence = new_speech_bubble_list[i]
                    sentiment_result = sentiment_nlp(sentence)
                    sentiment_score, positivity_value = map_sentiments(sentiment_result)
                    question_sentiment_list.append(sentiment_score)
                    net_sentiment_list.append(sentiment_score)

                    if positivity_value == "positive":
                        net_positive += 1
                        Qpositive += 1

                    elif positivity_value == "negative":
                        net_negative += 1
                        Qnegative += 1

                    else:
                        net_neutral += 1
                        Qneutral += 1
            except:
                pass
        
        for speech_bubble in management_speech:
            try:
                new_speech_bubble = ac.remove_words(speech_bubble)
                new_speech_bubble = re.sub('[^\S\n]+', ' ', new_speech_bubble)

                if new_speech_bubble[0] == " ": 
                    new_speech_bubble = new_speech_bubble.replace(" ", "", 1) # replace the first space bar with an empty string, for example ' is is string is is string' to 'is is string is is string'
                    
                # gets text complexity
                flesch_score = textstat.flesch_reading_ease(new_speech_bubble)
                reply_complex_list.append(flesch_score)
                net_text_complex_list.append(flesch_score)

                new_speech_bubble_list = split_paragraph_into_sentences(new_speech_bubble)

                fls_results = fls_nlp(new_speech_bubble_list)
                
                for i in range(0, len(new_speech_bubble_list)):
                    sentence = new_speech_bubble_list[i]
                    if fls_results[i]['label'] == 'Not FLS':
                        n_flslist.append(sentence)
                    elif fls_results[i]['label'] == 'Specific FLS':
                        s_flslist.append(sentence)
                    elif fls_results[i]['label'] == 'Non-specific FLS':                    
                        ns_flslist.append(sentence)
                    
            except:
                pass
                
        # for "n_flslist":
        n_fls_sentiment_list, reply_sentiment_list, Rpositive, Rnegative, Rneutral, NFLSpositive, NFLSnegative, NFLSneutral, net_positive, net_negative, net_neutral = get_SentimentLists_from_FLS(net_sentiment_list, n_flslist, n_fls_sentiment_list, reply_sentiment_list, Rpositive, Rnegative, Rneutral, NFLSpositive, NFLSnegative, NFLSneutral, net_positive, net_negative, net_neutral)
        
        # for "s_flslist":
        s_fls_sentiment_list, reply_sentiment_list, Rpositive, Rnegative, Rneutral, SFLSpositive, SFLSnegative, SFLSneutral, net_positive, net_negative, net_neutral = get_SentimentLists_from_FLS(net_sentiment_list, s_flslist, s_fls_sentiment_list, reply_sentiment_list, Rpositive, Rnegative, Rneutral, SFLSpositive, SFLSnegative, SFLSneutral, net_positive, net_negative, net_neutral)
        
        # for "ns_flslist":
        ns_fls_sentiment_list, reply_sentiment_list, Rpositive, Rnegative, Rneutral, NSFLSpositive, NSFLSnegative, NSFLSneutral, net_positive, net_negative, net_neutral = get_SentimentLists_from_FLS(net_sentiment_list, ns_flslist, ns_fls_sentiment_list, reply_sentiment_list, Rpositive, Rnegative, Rneutral, NSFLSpositive, NSFLSnegative, NSFLSneutral, net_positive, net_negative, net_neutral)

        feature_extract_23 = statistics.mean(net_sentiment_list)

        try:
            feature_extract_24 = net_positive/(net_positive+net_negative+net_neutral)
            feature_extract_25 = net_negative/(net_positive+net_negative+net_neutral)
        except:
            pass
        try:
            feature_extract_26 = statistics.mean(net_text_complex_list)
        except:
            pass
        try:
            feature_extract_27 = statistics.mean(question_sentiment_list)
        except:
            pass
        try:
            feature_extract_28 = Qpositive/(Qpositive+Qnegative+Qneutral)
            feature_extract_29 = Qnegative/(Qpositive+Qnegative+Qneutral)
        except:
            pass
        try:
            feature_extract_30 = statistics.mean(questions_complex_list)
        except:
            pass
        try:
            feature_extract_31 = statistics.mean(reply_sentiment_list)
        except:
            pass
        try:
            feature_extract_32 = Rpositive/(Rpositive+Rnegative+Rneutral)
            feature_extract_33 = Rnegative/(Rpositive+Rnegative+Rneutral)
        except:
            pass
        try:
            feature_extract_34 = statistics.mean(reply_complex_list)
        except:
            pass
        try:
            feature_extract_35 = statistics.mean(s_fls_sentiment_list)
        except:
            pass
        try:
            feature_extract_36 = SFLSpositive/(SFLSpositive+SFLSnegative+SFLSneutral)
            feature_extract_37 = SFLSnegative/(SFLSpositive+SFLSnegative+SFLSneutral)
        except:
            pass
        
        try:
            feature_extract_38 = statistics.mean(ns_fls_sentiment_list)
        except:
            pass
        try:
            feature_extract_39 = NSFLSpositive/(NSFLSpositive+NSFLSnegative+NSFLSneutral)
            feature_extract_40 = NSFLSnegative/(NSFLSpositive+NSFLSnegative+NSFLSneutral)
        except:
            pass
        
        try:
            feature_extract_41 = statistics.mean(n_fls_sentiment_list)
        except:
            pass
        try:
            feature_extract_42 = NFLSpositive/(NFLSpositive+NFLSnegative+NFLSneutral)
            feature_extract_43 = NFLSnegative/(NFLSpositive+NFLSnegative+NFLSneutral)
        except:
            pass

    except:
        pass
    return [feature_extract_23, feature_extract_24, feature_extract_25, feature_extract_26, feature_extract_27, feature_extract_28, feature_extract_29, feature_extract_30, feature_extract_31, feature_extract_32, feature_extract_33, feature_extract_34, feature_extract_35, feature_extract_36, feature_extract_37, feature_extract_38, feature_extract_39, feature_extract_40, feature_extract_41, feature_extract_42, feature_extract_43]

def get_SentimentLists_from_FLS(net_sentiment_list, THIS_flslist, THISfls_sentiment_list, reply_sentiment_list, Rpositive, Rnegative, Rneutral, FLSpositive, FLSnegative, FLSneutral, net_positive, net_negative, net_neutral):
    for each_fls_sentence in THIS_flslist:
        sentiment_result = sentiment_nlp(each_fls_sentence)
        sentiment_score, positivity_value = map_sentiments(sentiment_result)
        THISfls_sentiment_list.append(sentiment_score)
        reply_sentiment_list.append(sentiment_score)
        net_sentiment_list.append(sentiment_score)
        
        if positivity_value == "positive":
            net_positive += 1
            FLSpositive += 1
            Rpositive += 1
        elif positivity_value == "negative":
            net_negative += 1
            FLSnegative += 1
            Rnegative += 1

        else:
            net_neutral += 1
            FLSneutral += 1
            Rneutral += 1

    return THISfls_sentiment_list, reply_sentiment_list, Rpositive, Rnegative, Rneutral, FLSpositive, FLSnegative, FLSneutral, net_positive, net_negative, net_neutral

def deepCleanTranscript(mytranscript, speaker_names):
    updatedTranscript = ' '.join(mytranscript)
    new_speaker_names = [word + ':' for word in speaker_names]
    ac = AhoCorasick(new_speaker_names)

    updatedTranscript = ac.remove_words(updatedTranscript)
 
    if updatedTranscript[0] == " ": 
        updatedTranscript = updatedTranscript.replace(" ", "", 1)

    updatedTranscript = re.sub('[^\S\n]+', ' ', updatedTranscript)
    updatedTranscript.lower()

    return updatedTranscript

def getFeature44to73(mytranscript, speaker_names):
    marginSentimentList = []
    mar_positive = 0
    mar_negative = 0
    mar_neutral = 0

    costSentimentList = []
    cost_positive = 0
    cost_negative = 0
    cost_neutral = 0

    revenueSentimentList = []
    rev_positive = 0
    rev_negative = 0
    rev_neutral = 0

    earningsEBIDTASentimentList = []
    ear_positive = 0
    ear_negative = 0
    ear_neutral = 0
    
    growthSentimentList = []
    gro_positive = 0
    gro_negative = 0
    gro_neutral = 0

    leverageDebtSentimentList = []
    lev_positive = 0
    lev_negative = 0
    lev_neutral = 0

    IndSentimentList = []
    ind_positive = 0
    ind_negative = 0
    ind_neutral = 0

    operationSentimentList = []
    ope_positive = 0
    ope_negative = 0
    ope_neutral = 0

    cashflowSentimentList = []
    cash_positive = 0
    cash_negative = 0
    cash_neutral = 0

    dividendSentimentList = []
    div_positive = 0
    div_negative = 0
    div_neutral = 0

    feature_extract_44 = 0
    feature_extract_45 = 0
    feature_extract_46 = 0
    feature_extract_47 = 0
    feature_extract_48 = 0
    feature_extract_49 = 0
    feature_extract_50 = 0
    feature_extract_51 = 0
    feature_extract_52 = 0
    feature_extract_53 = 0
    feature_extract_54 = 0
    feature_extract_55 = 0
    feature_extract_56 = 0
    feature_extract_57 = 0
    feature_extract_58 = 0
    feature_extract_59 = 0
    feature_extract_60 = 0
    feautre_extract_61 = 0
    feature_extract_62 = 0
    feature_extract_63 = 0
    feature_extract_64 = 0
    feature_extract_65 = 0
    feature_extract_66 = 0
    feature_extract_67 = 0
    feature_extract_68 = 0
    feature_extract_69 = 0 
    feature_extract_70 = 0
    feature_extract_71 = 0
    feature_extract_72 = 0
    feature_extract_73 = 0

    updatedTranscript = deepCleanTranscript(mytranscript, speaker_names)
    
    updatedTranscriptList = split_paragraph_into_sentences(updatedTranscript)

    for mysentence in updatedTranscriptList:
        if (" margin" in mysentence) or (" return" in mysentence):
            sentiment_result = sentiment_nlp(mysentence)
            sentiment_score, positivity_value = map_sentiments(sentiment_result)
            marginSentimentList.append(sentiment_score)
            if positivity_value == "positive":
                mar_positive += 1

            elif positivity_value == "negative":
                mar_negative += 1

            else:
                mar_neutral += 1
            

        if " cost" in mysentence:
            sentiment_result = sentiment_nlp(mysentence)
            sentiment_score, positivity_value = map_sentiments(sentiment_result)
            costSentimentList.append(sentiment_score)
            if positivity_value == "positive":
                cost_positive += 1

            elif positivity_value == "negative":
                cost_negative += 1

            else:
                cost_neutral += 1

        if (" revenue" in mysentence) or (" top line" in mysentence) or (" sales" in mysentence):
            sentiment_result = sentiment_nlp(mysentence)
            sentiment_score, positivity_value = map_sentiments(sentiment_result)
            revenueSentimentList.append(sentiment_score)

            if positivity_value == "positive":
                rev_positive += 1

            elif positivity_value == "negative":
                rev_negative += 1

            else:
                rev_neutral += 1

        if (" earning" in mysentence) or (" EBIT" in mysentence) or (" profit" in mysentence) or (" bottom line" in mysentence) or (" net income" in mysentence):
            sentiment_result = sentiment_nlp(mysentence)
            sentiment_score, positivity_value = map_sentiments(sentiment_result)
            earningsEBIDTASentimentList.append(sentiment_score)
            if positivity_value == "positive":
                ear_positive += 1

            elif positivity_value == "negative":
                ear_negative += 1

            else:
                ear_neutral += 1

        if (" growth" in mysentence) or (" organic" in mysentence):
            sentiment_result = sentiment_nlp(mysentence)
            sentiment_score, positivity_value = map_sentiments(sentiment_result)
            growthSentimentList.append(sentiment_score)
            if positivity_value == "positive":
                gro_positive += 1

            elif positivity_value == "negative":
                gro_negative += 1

            else:
                gro_neutral += 1

        if (" leverage" in mysentence) or (" debt" in mysentence):
            sentiment_result = sentiment_nlp(mysentence)
            sentiment_score, positivity_value = map_sentiments(sentiment_result)
            leverageDebtSentimentList.append(sentiment_score)
            if positivity_value == "positive":
                lev_positive += 1

            elif positivity_value == "negative":
                lev_negative += 1

            else:
                lev_neutral += 1

        if (" industry" in mysentence) or (" industr" in mysentence) or (" economy" in mysentence) or (" economi”" in mysentence) or (" sector" in mysentence):
            sentiment_result = sentiment_nlp(mysentence)
            sentiment_score, positivity_value = map_sentiments(sentiment_result)
            IndSentimentList.append(sentiment_score)
            if positivity_value == "positive":
                ind_positive += 1

            elif positivity_value == "negative":
                ind_negative += 1

            else:
                ind_neutral += 1

        if " operation" in mysentence:
            sentiment_result = sentiment_nlp(mysentence)
            sentiment_score, positivity_value = map_sentiments(sentiment_result)
            operationSentimentList.append(sentiment_score)
            if positivity_value == "positive":
                ope_positive += 1

            elif positivity_value == "negative":
                ope_negative += 1

            else:
                ope_neutral += 1
        
        if (" cashflow" in mysentence) or (" cash flow" in mysentence):
            sentiment_result = sentiment_nlp(mysentence)
            sentiment_score, positivity_value = map_sentiments(sentiment_result)
            cashflowSentimentList.append(sentiment_score)
            if positivity_value == "positive":
                cash_positive += 1

            elif positivity_value == "negative":
                cash_negative += 1

            else:
                cash_neutral += 1

        if (" dividend" in mysentence) or (" buyback" in mysentence) or (" repurchase" in mysentence):
            sentiment_result = sentiment_nlp(mysentence)
            sentiment_score, positivity_value = map_sentiments(sentiment_result)
            dividendSentimentList.append(sentiment_score)
            if positivity_value == "positive":
                div_positive += 1

            elif positivity_value == "negative":
                div_negative += 1

            else:
                div_neutral += 1

    try:
        feature_extract_44 = statistics.mean(marginSentimentList)
    except:
        pass
    try:
        feature_extract_45 = mar_positive/(mar_positive+mar_negative+mar_neutral)
        feature_extract_46 = mar_negative/(mar_positive+mar_negative+mar_neutral)
    except:
        pass
    try:
        feature_extract_47 = statistics.mean(costSentimentList)
    except:
        pass
    try:
        feature_extract_48 = cost_positive/(cost_positive+cost_negative+cost_neutral)
        feature_extract_49 = cost_negative/(cost_positive+cost_negative+cost_neutral)
    except:
        pass
    try:
        feature_extract_50 = statistics.mean(revenueSentimentList)
    except:
        pass
    try:
        feature_extract_51 = rev_positive/(rev_positive+rev_negative+rev_neutral)
        feature_extract_52 = rev_negative/(rev_positive+rev_negative+rev_neutral)
    except:
        pass
    try:
        feature_extract_53 = statistics.mean(earningsEBIDTASentimentList)
    except:
        pass
    try:
        feature_extract_54 = ear_positive/(ear_positive+ear_negative+ear_neutral)
        feature_extract_55 = ear_negative/(ear_positive+ear_negative+ear_neutral)
    except:
        pass

    try:
        feature_extract_56 = statistics.mean(growthSentimentList)
    except:
        pass
    try:
        feature_extract_57 = gro_positive/(gro_positive+gro_negative+gro_neutral)
        feature_extract_58 = gro_negative/(gro_positive+gro_negative+gro_neutral)
    except:
        pass

    try:
        feature_extract_59 = statistics.mean(leverageDebtSentimentList)
    except:
        pass
    try:
        feature_extract_60 = lev_positive/(lev_positive+lev_negative+lev_neutral)
        feature_extract_61 = lev_negative/(lev_positive+lev_negative+lev_neutral)
    except:
        pass

    try:
        feature_extract_62 = statistics.mean(IndSentimentList)
    except:
        pass
    try:
        feature_extract_63 = ind_positive/(ind_positive+ind_negative+ind_neutral)
        feature_extract_64 = ind_negative/(ind_positive+ind_negative+ind_neutral)
    except:
        pass

    try:
        feature_extract_65 = statistics.mean(operationSentimentList)
    except:
        pass
    try:
        feature_extract_66 = ope_positive/(ope_positive+ope_negative+ope_neutral)
        feature_extract_67 = ope_negative/(ope_positive+ope_negative+ope_neutral)

    except:
        pass

    try:
        feature_extract_68 = statistics.mean(cashflowSentimentList)
    except:
        pass
    try:
        feature_extract_69 = cash_positive/(cash_positive+cash_negative+cash_neutral)
        feature_extract_70 = cash_negative/(cash_positive+cash_negative+cash_neutral)

    except:
        pass

    try:
        feature_extract_71 = statistics.mean(dividendSentimentList)
    except:
        pass
    try:
        feature_extract_72 = div_positive/(div_positive+div_negative+div_neutral)
        feature_extract_73 = div_negative/(div_positive+div_negative+div_neutral)
    except:
        pass

    return [feature_extract_44, feature_extract_45, feature_extract_46, feature_extract_47, feature_extract_48, feature_extract_49, feature_extract_50, feature_extract_51, feature_extract_52, feature_extract_53, feature_extract_54, feature_extract_55, feature_extract_56, feature_extract_57, feature_extract_58, feature_extract_59, feature_extract_60, feautre_extract_61, feature_extract_62, feature_extract_63, feature_extract_64, feature_extract_65, feature_extract_66, feature_extract_67, feature_extract_68, feature_extract_69, feature_extract_70, feature_extract_71, feature_extract_72, feature_extract_73]

def map_sentiments(sentiment_result):
    sentiment_result = sentiment_result[0]
    if sentiment_result['label'] == 'Negative':
        return -1 * sentiment_result['score'], "negative"
    
    elif sentiment_result['label'] == 'Neutral':
        return 0, "neutral"
    
    elif sentiment_result['label'] == 'Positive':
        return sentiment_result['score'], "positive"


def split_paragraph_into_sentences(temp):
    sentences = nltk.sent_tokenize(temp)
    return sentences

def get_NLP_values(liststr):
    # further analysis includes finding sentiment and word complexity.
    if len(liststr) == 0:
        return 0, 0, 0
    
    else:
        # maps sentiment data so it outputs a single sentiment value
        sentiment_result = sentiment_nlp(liststr)
        # gets 
        sentiment_score = map_sentiments(sentiment_result)

        # word complexity:
        flesch_score = textstat.flesch_reading_ease(liststr)
        gunning_fog_score = textstat.gunning_fog(liststr)

        return sentiment_score, flesch_score, gunning_fog_score
        
